In [1]:
import pandas as pd
import numpy as np
import re
import json
from random import sample
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

# Load Data

In [43]:
# VAERS tables
data = pd.read_csv('./VAERS22/2022VAERSDATA.csv', encoding = 'Windows-1252')
vax = pd.read_csv('./VAERS22/2022VAERSVAX.csv', encoding = 'Windows-1252')
symp = pd.read_csv('./VAERS22/2022VAERSSYMPTOMS.csv', encoding = 'Windows-1252')

vaers_fields = {}
for x in list(data):
    vaers_fields[x] = 'DATA'
for x in list(vax):
    if x not in vaers_fields.keys():
        vaers_fields[x] = 'VAX'
for x in list(symp):
    if x not in vaers_fields.keys():
        vaers_fields[x] = 'SYMPTOMS' 
        
# load lookup values
f = open('./VAERS22/lookup/lookup22.json')
data = json.load(f)

lookup22 = {}
for x in data.keys():
    if x in ['SYMPTOM1', 'SYMPTOMVERSION1', 'SYMPTOM2', 'SYMPTOMVERSION2', 'SYMPTOM3', 'SYMPTOMVERSION3', 'SYMPTOM4', 'SYMPTOMVERSION4', 'SYMPTOM5', 'SYMPTOMVERSION5', 'SYMPTOM', 'SYMPTOM_TEXT']:
        lookup22[x] = {'query_field': 'SYMPTOMS', 'values': data[x]}
    elif (x == 'PRIOR_VAX_DATE'): 
        lookup22['PRIOR_VAX_DATE'] = {'query_field': 'DATA.PRIOR_VAX', 'values': data[x]}
    else: 
        temp = vaers_fields[x] + '.' + x
        lookup22[x] = {'query_field': temp, 'values': data[x]}

# load question templates
easy_qst = pd.read_excel("./QuestionTemplate/Template-Easy-Text2ESQ.xlsx")
med_qst = pd.read_excel("./QuestionTemplate/Template-Medium-Text2ESQ.xlsx")

# load query templates
search_templates = pd.read_excel('queryTemplates.xlsx', index_col='ID')

C:\Users\eyang\AppData\Local\Temp\ipykernel_34512\3090378548.py:2: DtypeWarning: Columns (7,12,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./VAERS22/2022VAERSDATA.csv', encoding = 'Windows-1252')


# Functions

In [90]:
def find_key(tokens, pattern):
    key_idx = []
    for idx, x in enumerate(tokens):
        token_key = re.findall(pattern, x)
        if token_key != []:
            key_idx.append((token_key[0], idx))
    return key_idx

# def randsample(keys_tmpl, lookup_table):
#     out = []
#     vLen = len(lookup_table[keys_tmpl[0][0]]['values'])
#     if vLen <= 100:
#         for x in lookup_table[keys_tmpl[0][0]]['values']:
#             temp = [{'field': keys_tmpl[0][0], 'query_field': lookup_table[keys_tmpl[0][0]]['query_field'], 'values': x}]
#             out.append(temp)
#     elif vLen > 100:
#         for x in range(100):
#             temp = [{'field': keys_tmpl[0][0], 'query_field':lookup_table[keys_tmpl[0][0]]['query_field'], 'values':sample(lookup_table[keys_tmpl[0][0]]['values'], 1)[0]}]
#             out.append(temp)
#     return out

def create_mini_lookup(keys_tmpl, lookup_table, num=150):
    out = []
    for x in range(num):
        temp = [{'field': keys_tmpl[x][0], 'query_field':lookup_table[keys_tmpl[x][0]]['query_field'], 'values':sample(lookup_table[keys_tmpl[x][0]]['values'], 1)[0]} for x in range(len(keys_tmpl))]
        out.append(temp)

    return out

In [116]:
def BIO_tagging(question, values_tmpl, keys_tmpl, token_tmpl):
    
    BIO = []
    token_question = word_tokenize(question)
    v_len = [len(word_tokenize(str(x['values']))) for x in values_tmpl]
    BIO = ['O' for x in range(keys_tmpl[0][1])]
    n = 0
    for idx, l in enumerate(v_len):
        for x in range(l):
            if x == 0:
                beginning = 'B-{0}'.format(keys_tmpl[idx][0])
                BIO.append(beginning)
            else: 
                inside = 'I-{0}'.format(keys_tmpl[idx][0])
                BIO.append(inside)
        try:
            ph_idx_diff = keys_tmpl[idx+1][1] - keys_tmpl[idx][1] - n
            if ph_idx_diff > 1:
                for x in range(ph_idx_diff -1):
                    BIO.append("O")
        except:
            sen_idx_diff = len(token_tmpl) - 1 - keys_tmpl[idx][1]
            if sen_idx_diff > 0:
                for x in range(sen_idx_diff):
                    BIO.append("O")
                    
    output = []
    for x in list(zip(token_question, BIO)):
        temp_dict = {'text': x[0], 'label':x[1]}
        output.append(temp_dict)
            
    return output


def sub_BIO(q, lookup_table):
    pattern = re.compile(r'\[(.*?)\]', re.S)
    questions= []
    tokens = []
    
    tokenizer = RegexpTokenizer('\w+|\[\w+\]|\S+')
    token_tmpl = tokenizer.tokenize(q)
    keys_tmpl = find_key(token_tmpl, pattern) #[(placeholder1, idx1), (placeholder2, idx2)]
    
    # if len(keys_tmpl) == 1:
    #     mini_lookup = randsample(keys_tmpl, lookup_table)
    #     for v in mini_lookup:
    #         question = re.sub(r"\[(.*?)\]", lambda x: str(v[0]['values']), q)
    #         questions.append(question)
    #         token_bio = BIO_tagging(question, v, keys_tmpl, token_tmpl)
    #         tokens.append(token_bio)
            
    # elif len(keys_tmpl) > 1:
    mini_lookup = create_mini_lookup(keys_tmpl, lookup_table)
    try:
        for vals in mini_lookup:
            for idx, val in enumerate(vals):
                if idx == 0:
                    temp = re.sub(r"\[(.*?)\]", lambda x: str(val['values']), q, count=1)
                else:
                    question = re.sub(r"\[(.*?)\]", lambda x: str(val['values']), temp, count=1)
                    temp = question
            questions.append(temp)
            token_bio = BIO_tagging(temp, vals, keys_tmpl, token_tmpl)
            tokens.append(token_bio)
        output = {'Questions':questions, 'BIO_tagging':tokens, 'lookup': mini_lookup}
    except:
        output = {'Questions':np.nan, 'BIO_tagging':np.nan, 'lookup':np.nan}
    

    return output


def generate_query(df):
    out = []
    for idx in range(len(df)):
        query = df['query_get_tp'][idx]
        query = re.sub(r"\[id\]", lambda x: str(df['Query Template ID'][idx]), query)
        temp = query
        if (df['Query Template ID'][idx] in [14, 15, 16, 17]):
            for value in df['lookup'][idx]:
                if value['field'] in ['AGE_YRS', 'VAX_DATE']:
                    temp = re.sub(r"\[field_1\]", lambda x: value['query_field'], temp)
                    temp = re.sub(r"\[(.*?)\]", lambda x: str(value['values']), temp, count=1)
                else: 
                    temp = re.sub(r"\[field\_[23]\]", lambda x: value['query_field'], temp, count=1)
                    temp = re.sub(r"\[value\_[123]\]", lambda x: str(value['values']), temp, count=1)
        else:
            for value in df['lookup'][idx]: 
                temp = re.sub(r"\[field\_[123]\]", lambda x: value['query_field'], temp, count=1)
                temp = re.sub(r"\[value\_[123]\]", lambda x: str(value['values']), temp, count=1)

        out.append(temp)
    return out
                   

In [145]:
def create_id(question, query, index):
    
    # generate id with a format of "Question template id" + "Query template id" + "Index"
    # example: "01100001" 
    # the first two digits "01" indicates the question template id 1
    # the third digit "1" indicates the query template id 1
    # the last five digits "00001" indicates the index of the question 1
    
    question = str(question)
    query = str(query)
    index = str(index)
    
    if len(question) == 1:
        question = '0' + question
    
    diff = 5 - len(index)
    if diff < 5:
        index = '0'*diff + index
    
    output = question + query + index
    
    return output

def populate_questions(dataset, lookup_table):
    df = dataset.copy()
    df['Question Template'] = df['Question Template'].apply(lambda x: x.split("##"))
    df['query_post'] = df['Query Template ID'].apply(lambda x: search_templates.iloc[x-1, 0])
    df['query_get_tp'] = df['Query Template ID'].apply(lambda x: search_templates.iloc[x-1, 1])
    df = df.explode('Question Template').reset_index().drop(columns='index')
    
    df['output'] = df['Question Template'].apply(lambda x: sub_BIO(x, lookup_table))
    df['Questions'] = df.output.apply(lambda x: x['Questions'])
    df['Tokens'] = df.output.apply(lambda x: x['BIO_tagging'])
    df['lookup'] = df.output.apply(lambda x: x['lookup'])
    df = df.drop(columns='output')
    df = df.apply(lambda x: x.explode()).reset_index()

    df = df.dropna()
    df = df.loc[df.astype(str).drop_duplicates().index]
    df = df.reset_index(drop=True)
    queries = generate_query(df)
    df['query_get'] = queries

    df['index'] = df.index.values
    df['Query Template ID'] = df['Query Template ID'].apply(lambda x: int(x))
    df['id'] = list(map(lambda x, y, z: create_id(x, y, z), \
                        df['Question Template ID'], \
                        df['Query Template ID'], \
                        df['index']))
    df = df.drop(columns='index')

    return df

# Final

In [11]:
medium = populate_questions(med_qst, lookup22)
medium.to_csv('./VAERS22/final/medium.csv')

In [146]:
easy = populate_questions(easy_qst, lookup22)
easy.to_csv('./VAERS22/final/easy.csv')

In [147]:
easy

,Question Template ID,Data File,Field,Query Field,Question Template,Query Template ID,Difficulty,query_post,query_get_tp,Questions,Tokens,lookup,query_get,id
0,1,VAERSDATA,RECVDATE,DATA.RECVDATE,Give me all the patients whose information are...,1,Easy,"POST _scripts/1 {""script"": {""lang"": ""mustache""...","GET _search/template {""id"": ""[id]"", ""params"": ...",Give me all the patients whose information are...,"[{'text': 'Give', 'label': 'O'}, {'text': 'me'...","[{'field': 'RECVDATE', 'query_field': 'DATA.RE...","GET _search/template {""id"": ""1"", ""params"": {""f...",01100000
1,1,VAERSDATA,RECVDATE,DATA.RECVDATE,Give me all the patients whose information are...,1,Easy,"POST _scripts/1 {""script"": {""lang"": ""mustache""...","GET _search/template {""id"": ""[id]"", ""params"": ...",Give me all the patients whose information are...,"[{'text': 'Give', 'label': 'O'}, {'text': 'me'...","[{'field': 'RECVDATE', 'query_field': 'DATA.RE...","GET _search/template {""id"": ""1"", ""params"": {""f...",01100001
2,1,VAERSDATA,RECVDATE,DATA.RECVDATE,Give me all the patients whose information are...,1,Easy,"POST _scripts/1 {""script"": {""lang"": ""mustache""...","GET _search/template {""id"": ""[id]"", ""params"": ...",Give me all the patients whose information are...,"[{'text': 'Give', 'label': 'O'}, {'text': 'me'...","[{'field': 'RECVDATE', 'query_field': 'DATA.RE...","GET _search/template {""id"": ""1"", ""params"": {""f...",01100002
3,1,VAERSDATA,RECVDATE,DATA.RECVDATE,Give me all the patients whose information are...,1,Easy,"POST _scripts/1 {""script"": {""lang"": ""mustache""...","GET _search/template {""id"": ""[id]"", ""params"": ...",Give me all the patients whose information are...,"[{'text': 'Give', 'label': 'O'}, {'text': 'me'...","[{'field': 'RECVDATE', 'query_field': 'DATA.RE...","GET _search/template {""id"": ""1"", ""params"": {""f...",01100003
4,1,VAERSDATA,RECVDATE,DATA.RECVDATE,Give me all the patients whose information are...,1,Easy,"POST _scripts/1 {""script"": {""lang"": ""mustache""...","GET _search/template {""id"": ""[id]"", ""params"": ...",Give me all the patients whose information are...,"[{'text': 'Give', 'label': 'O'}, {'text': 'me'...","[{'field': 'RECVDATE', 'query_field': 'DATA.RE...","GET _search/template {""id"": ""1"", ""params"": {""f...",01100004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13035,45,VAERSSYMPTOMS,SYMPTOM1，SYMPTOM2，SYMPTOM3,SYMPTOMS,which [SYMPTOM] is the most common after vac...,3,Easy,"POST _scripts/3 {""script"": {""lang"": ""mustache""...","GET _search/template {""id"": ""[id]"", ""params"": ...",which Peritoneal disorder is the most common...,"[{'text': 'which', 'label': 'O'}, {'text': 'Pe...","[{'field': 'SYMPTOM', 'query_field': 'SYMPTOMS...","GET _search/template {""id"": ""3"", ""params"": {""f...",45313035
13036,45,VAERSSYMPTOMS,SYMPTOM1，SYMPTOM2，SYMPTOM3,SYMPTOMS,which [SYMPTOM] is the most common after vac...,3,Easy,"POST _scripts/3 {""script"": {""lang"": ""mustache""...","GET _search/template {""id"": ""[id]"", ""params"": ...",which Glucose urine present is the most comm...,"[{'text': 'which', 'label': 'O'}, {'text': 'Gl...","[{'field': 'SYMPTOM', 'query_field': 'SYMPTOMS...","GET _search/template {""id"": ""3"", ""params"": {""f...",45313036
13037,45,VAERSSYMPTOMS,SYMPTOM1，SYMPTOM2，SYMPTOM3,SYMPTOMS,which [SYMPTOM] is the most common after vac...,3,Easy,"POST _scripts/3 {""script"": {""lang"": ""mustache""...","GET _search/template {""id"": ""[id]"", ""params"": ...",which Cardiac imaging procedure is the most ...,"[{'text': 'which', 'label': 'O'}, {'text': 'Ca...","[{'field': 'SYMPTOM', 'query_field': 'SYMPTOMS...","GET _search/template {""id"": ""3"", ""params"": {""f...",45313037
13038,45,VAERSSYMPTOMS,SYMPTOM1，SYMPTOM2，SYMPTOM3,SYMPTOMS,which [SYMPTOM] is the most common after vac...,3,Easy,"POST _scripts/3 {""script"": {""lang"": ""mustache""...","GET _search/template {""id"": ""[id]"", ""params"": ...",which Foetal growth abnormality is the mos